In [1]:
import numpy as np
import matplotlib.pyplot as plt

import matplotlib.style
import matplotlib as mpl
mpl.style.use('classic')

import baraffe_tables
from baraffe_tables.table_search import baraffe_table_search

from astropy.table import Table
from astropy import units as u
from astropy.constants import G

import mesa_helper as mh
import os
import shutil

%matplotlib inline

In [2]:
mJtomSun = u.jupiterMass.to(u.solMass)
mJtoGrams = u.jupiterMass.to(u.g)

rJtorSun = u.jupiterRad.to(u.solRad)
rJtoCm = u.jupiterRad.to(u.cm)

print(mJtomSun)
print(mJtoGrams)
print(rJtorSun)
print(rJtoCm)


0.0009545942339693249
1.8981245973360504e+30
0.10276268506540176
7149200000.0


In [3]:
#jupiter masses
#Minit = np.array((20.0, 14.0, 9.8, 6.9, 4.8, 3.4, 2.3, 1.6, 1.1, 0.82, 0.57, 0.4, 0.28, 0.2))
Minit = np.atleast_1d(np.array((0.2)))

#jupiter radii
Rinit = np.array((5.0, 2.0, 1.5, 1.0))
#Rinit = np.array((2.0, 1.75, 1.5, 1.25, 1.0, 0.75, 0.5, 0.25, 0.1))
#Rinit = np.array((2.0, 1.5, 1.25, 1.0))

In [4]:
# make directories for all planet initial masses
os.chdir('/Users/Emily/Documents/astro/giant_planets/MESAtests/reproducePaxton+2011fig18_r12115')
    
for i, m in enumerate(Minit):
    os.system("rm -r Minit={0}Mj".format(m))
    os.system("cp -r template_makeplanet_r12115 Minit={0}Mj".format(m))

#edit inlist_create, inlist_evolve, rn
for i, m in enumerate(Minit):
    print(m)
    #edit inlists
    create_model_filename = "planet_{0}_Mj_{1}_Rj.mod".format(m, Rinit[0])

    inlist_create = mh.Inlist('./Minit={0}Mj/inlist_make_planet'.format(m))

    #edit inlist_create options
    inlist_create.set_option('mass_in_gm_for_create_initial_model', m*mJtoGrams)
    # start with initial radius equal to 5 Rj. This will need to be revised downward for less massive planets.
    # If it needs to be revised to be very small, may also need to change kap_lowT_prefix option.
    inlist_create.set_option('radius_in_cm_for_create_initial_model', Rinit[0]*rJtoCm)
    inlist_create.set_option('save_model_filename', create_model_filename)
    
    # execute mk script
    os.chdir('./Minit={0}Mj/'.format(m))
    os.system('./mk'.format(m))
    os.chdir('..')


0.2
Set mass_in_gm_for_create_initial_model to 3.79625d29
Set radius_in_cm_for_create_initial_model to 3.5746d10
Set save_model_filename to 'planet_0.2_Mj_5.0_Rj.mod'
gfortran -fopenmp -o ../star run_star_extras.o run_star.o  run.o  -L/Users/Emily/mesa-r12115/lib -lstar -lgyre -lionization -latm -lcolors -lnet -leos -lkap -lrates -lneu -lchem -linterp_2d -linterp_1d -lnum -lmtx -lconst -lutils -lf2crlibm -lcrlibm  `mesasdk_lapack_link` `mesasdk_blas_link` `mesasdk_hdf5_link`  `mesasdk_pgplot_link` -lz 


In [5]:
def update_radius(m, rold, rnew):
    """
    m in Mj, rnew in Rj
    Assumes that this will be run from within the relevant model folder
    """
    
    #edit inlists
    old_create_model_filename = "planet_{0}_Mj_{1}_Rj.mod".format(m, rold)
    
    create_model_filename = "planet_{0}_Mj_{1}_Rj.mod".format(m, rnew)

    inlist_create = mh.Inlist('./inlist_make_planet')

    #edit inlist_create options
    inlist_create.set_option('mass_in_gm_for_create_initial_model', m*mJtoGrams)
    inlist_create.set_option('radius_in_cm_for_create_initial_model', rnew*rJtoCm)
    inlist_create.set_option('save_model_filename', create_model_filename)

    return


In [6]:
pwd

'/Users/Emily/Documents/astro/giant_planets/MESAtests/reproducePaxton+2011fig18_r12115'

In [7]:
# need to try stepping through R values until code runs
for i, m in enumerate(Minit):
    os.chdir('/Users/Emily/Documents/astro/giant_planets/MESAtests/reproducePaxton+2011fig18_r12115')
    
    print("")
    print("m is {0} Mj".format(m))
    os.system('pwd')
    os.chdir('./Minit={0}Mj/'.format(m))
    os.system('pwd')

    print("r is {0} Rj".format(Rinit[0]))
    create_model_filename = "planet_{0}_Mj_{1}_Rj.mod".format(m, Rinit[0])

    os.system('./rn')

    #test if "create" and "evolve" models exist, indicating success
    test_create = os.path.isfile(create_model_filename)

    print("test_make is {0}".format(test_create))

    if test_create is True:
        #success, no need to try further r values
        continue
    else:
        print("model did not converge; stepping through decreasing radii")
        for j, r in enumerate(Rinit[:-1]):
            os.system('rm ./LOGS/*')
            os.system('rm ./photos/*')

            print("trying at new radius {0} Rj".format(Rinit[j+1]))
            update_radius(m, r, Rinit[j+1])
            create_model_filename = "planet_{0}_Mj_{1}_Rj.mod".format(m, Rinit[j+1])
            
            os.system('./rn')
            test_create = os.path.isfile(create_model_filename)
            
            if test_create is True:
                #success, no need to try further r values
                print("test_create is {0} ".format(test_create))
                break 
            
    


m is 0.2 Mj
/Users/Emily/Documents/astro/giant_planets/MESAtests/reproducePaxton+2011fig18_r12115
/Users/Emily/Documents/astro/giant_planets/MESAtests/reproducePaxton+2011fig18_r12115/Minit=0.2Mj
r is 5.0 Rj
DATE: 2024-08-08
TIME: 12:09:02
                                         version_number       12115
 read inlist_make_planet
 create initial model
 set_some_eos_flags
 use_eosPTEH_for_low_density T
 use_eosPTEH_for_high_Z T
 use_eosELM T
                         create initial model iteration           1
   log10_cr(L/Lsun)=  -2.43507830E+00
   log10_cr(L/Lsun)=  -2.43364673E+00
   log10_cr(L/Lsun)=  -2.35489506E+00
                         create initial model iteration           2
   log10_cr(L/Lsun)=  -2.89839959E+00
   log10_cr(L/Lsun)=  -2.78553299E+00
   log10_cr(L/Lsun)=  -2.89924048E+00
                         create initial model iteration           3
   log10_cr(L/Lsun)=  -4.13477584E+00
   log10_cr(L/Lsun)=  -4.13337028E+00
   log10_cr(L/Lsun)=  -4.02970149E+00
       